In [128]:
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
# We start by getting access to the drive

In [129]:
import pandas as pd

train_df = pd.read_csv("../train.csv")
test_df = pd.read_csv("../test.csv")

Vamos a ver qué tal se ha cargado

## Eliminando atributos no correctos

Vamos a eliminar del conjunto de entrenamiento los atributos 'Artista' y
'Titulo' porque no se puede aprender a partir de ellos.

También quitamos el atributo a predecir para evitar usarlo de entrada.

In [130]:
train_target = train_df.Category
targetencoder = LabelEncoder()
train_target = targetencoder.fit_transform(train_target)
train_df.drop(['Artista', 'Titulo', 'Category'], axis=1, inplace=True)

In [131]:
i=0
duplicados=[]
for a in train_df.duplicated(keep=False):
    if a:
        duplicados.append(i)
    i=i+1

In [132]:
train_df.drop(duplicados, axis=0, inplace=True)
train_df

,bailable,energia,clave,hablado,positivismo,duración en min/ms,acustico,vivacidad,popularidad,tempo,modo,volumen,ritmo,instrumental
0,63.1,7.93,P1.0,0.0468,0.6480,189373.000000,0.0218,9.76,63.0,13696.0,menor,-40.86,4,0.000565
1,70.6,6.56,P9.0,0.2900,0.8200,140564.000000,3.4000,16.30,48.0,16563.9,menor,-41.77,4,0.001670
2,81.5,6.61,P10.0,0.0996,0.6990,167787.000000,10.6000,28.40,54.0,12499.1,mayor,-82.08,3,2.660000
3,14.9,9.97,P1.0,0.1600,0.0954,295507.000000,0.1940,7.32,29.0,9144.7,menor,-23.49,4,0.009450
4,66.5,7.07,P11.0,0.0300,0.3910,156331.000000,19.3000,21.90,72.0,14495.2,mayor,-51.44,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14390,25.2,0.34,P3.0,0.0407,0.1470,5.535550,97.1000,11.20,37.0,11801.0,mayor,-292.98,3,9.280000
14391,57.2,6.28,P11.0,0.0397,0.0359,296013.000000,41.9000,23.50,36.0,11998.9,menor,-97.11,4,8.870000
14392,48.5,3.07,P1.0,0.0565,0.1320,4.300000,66.3000,12.30,47.0,8017.1,mayor,-131.29,4,0.000413
14393,34.3,5.32,P6.0,0.0318,0.4760,5.583433,93.4000,31.60,25.0,10477.5,mayor,-73.47,4,0.004650


In [133]:
train_target = np.delete(train_target, duplicados)
len(train_target)

12158

## Viendo nulos

Ahora voy a ver los nulos y procesarlos. En general hay que procesarlos mejor
pero aquí únicamente voy a reemplazarlos por las medias.

In [134]:
train_df.replace({'P1.0': 1.0, 'P9.0': 9.0, 'P10.0':10.0, 'P11.0':11.0, 'P2.0': 2.0, 'P6.0': 6.0, 'P4.0': 4.0,
       'P7.0': 7.0, 'P5.0': 5.0 , 'P8.0': 8.0, 'P3.0' : 3.0, 'menor': 0, 'mayor': 1}, inplace=True)
test_df.replace({'P1.0': 1.0, 'P9.0': 9.0, 'P10.0':10.0, 'P11.0':11.0, 'P2.0': 2.0, 'P6.0': 6.0, 'P4.0': 4.0,
       'P7.0': 7.0, 'P5.0': 5.0 , 'P8.0': 8.0, 'P3.0' : 3.0, 'menor': 0, 'mayor': 1}, inplace=True)

In [135]:
ids = test_df['id']
test_df.drop(columns="id", inplace=True)


In [120]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier
import xgboost as xgb

PARAMETERS = {#'learning_rate': np.linspace(0.25, 0.35, 3)
#"n_estimators":[160, 140, 120],
#"max_depth": np.arange(2, 15, 1)
#"min_child_weight":[0.4, 0.3, 0.2],
#"learning_rate":[ 0.088, 0.083]}
}
estimator = xgb.XGBClassifier(random_state=0, learning_rate=0.088, max_depth=5,colsample_bytree=0.75, min_child_weight=4, reg_alpha=0.001)

model = GridSearchCV(estimator=estimator, param_grid=PARAMETERS, cv=5, n_jobs=-1, scoring="accuracy")

model.fit(train_df, train_target)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=0.75,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=0.088, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=5,
                                     max_l

In [121]:
model.best_score_, model.best_params_

(0.5982085493840525, {})

In [70]:
estimator.get_params()

{'categorical_features': None,
 'early_stopping': 'auto',
 'l2_regularization': 0.0020612244897959186,
 'learning_rate': 0.08336734693877551,
 'loss': 'auto',
 'max_bins': 255,
 'max_depth': 17,
 'max_iter': 100,
 'max_leaf_nodes': 20,
 'min_samples_leaf': 28,
 'monotonic_cst': None,
 'n_iter_no_change': 10,
 'random_state': 8,
 'scoring': 'loss',
 'tol': 1e-07,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [ ]:
0.08336734693877551

In [104]:
import sklearn
from sklearn.impute import SimpleImputer

imputer1 = SimpleImputer(strategy='mean')
train_df = imputer1.fit_transform(train_df)
test_df = imputer1.transform(test_df)

In [123]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

PARAMETERS = {#'max_features': [2, 3, 4, 5, 6, 7, 8, 9, 10],
#"colsample_bytree":[0.71, 0.75, 0.73, 0.74, 0.76],
#"max_depth": np.arange(2, 11, 1)
#"min_child_weight":[4, 5, 6, 7, 8, 9],
#"learning_rate":[ 0.088, 0.083]}
}
estimator = GradientBoostingClassifier(random_state=3, max_depth=12, n_estimators=325, min_samples_split=22, max_features=7)
model = GridSearchCV(estimator=estimator, param_grid=PARAMETERS, cv=5, n_jobs=-1, scoring="accuracy")

model.fit(train_df, train_target)

/home/nacho/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/nacho/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/nacho/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py", line 486, in fit
    X, y = self._validate_data(
  File "/home/nacho/anaconda3/lib/python3.9/site-packages/sklearn/base.py", line 581, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/home/nacho

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [148]:
model.best_score_, model.best_params_

(0.5322637103932693, {'max_features': 7})

In [7]:
from autoPyTorch import AutoNetClassification

ModuleNotFoundError: No module named 'autoPyTorch'

In [ ]:
autoPyTorch = AutoNetClassification
autoPyTorch = AutoNetClassification("tiny_cs", max_runtime=300, min_budget=30, max_budget=90)
autoPyTorch.fit(X_train, y_train, validation_split=0.3)
y_pred = autoPyTorch.predict(X_test)
print("Accuracy score", sklearn.metrics.accuracy_score(y_test, y_pred))

In [49]:
calibrated_clf.score(train_df, train_target)

0.6277002153226366

## Cambiando escala los datos numéricos

Vamos a aplicar una escala de los datos numéricos, dicho escalado es mejorable,
pero por ahora nos vale.

In [173]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(train_df)
X_train = scaler.transform(train_df)
y_train = train_target
X_test = scaler.transform(test_df)


## Aplicando el modelo

Una vez que ya está preprocesados los datos, voy a empezar a aplicar el modelo.

Voy a aplicar un modelo básico, el DecisionTreeClassifier

In [40]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

PARAMETERS = { #'random_state' : np.arange(4, 12, 1)
    #"colsample_bytree":[0.71, 0.75, 0.73, 0.74, 0.76],
#"max_depth": np.arange(2, 11, 1)
#"min_samples_split":[2, 3, 4, 5, 6, 7, 8, 9, 15, 20],
#"learning_rate":[0.058, 0.059, 0.0615, 0.0625, 0.06, 0.5, 0.25]
}
estimator = GradientBoostingClassifier(random_state=3, max_depth=2, n_estimators=600, min_samples_split=5, max_features=2)
model = GridSearchCV(estimator=estimator, param_grid=PARAMETERS, cv=5, n_jobs=-1, scoring="accuracy")

model.fit(train_df, train_target)

GridSearchCV(cv=5,
             estimator=GradientBoostingClassifier(max_depth=2, max_features=2,
                                                  min_samples_split=5,
                                                  n_estimators=600,
                                                  random_state=3),
             n_jobs=-1,
             param_grid={'random_state': array([ 4,  5,  6,  7,  8,  9, 10, 11])},
             scoring='accuracy')

In [42]:
model.best_score_, model.best_params_

(0.5387931525220948, {'random_state': 8})

In [182]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=31)
clf.fit(X_train, y_train)
scores = cross_val_score(clf, X_train, y_train, cv=5, n_jobs=-1)
scores.mean()

0.4443984938829068

In [31]:
from sklearn.linear_model import RidgeClassifierCV
clf = RidgeClassifierCV(alphas=[1e-3, 1e-2, 1e-1, 1], cv=5).fit(train_df, train_target)



In [32]:
clf.best_score_

0.44106364092470374

In [100]:
clf = HistGradientBoostingClassifier(random_state=8, l2_regularization=0.0020612244897959186, learning_rate=0.026000000000000002,
  max_depth=22, max_leaf_nodes=20, min_samples_leaf=28, class_weight='balanced')
clf.fit(train_df, train_target)
clf.score(train_df, train_target)

0.5816489546433284

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()

In [ ]:
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('gaussiannb', GaussianNB())])

Voy a validar cómo de bueno es usando validación cruzada con 5 conjuntos.

In [124]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb

pipehist = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),('estimator', HistGradientBoostingClassifier(random_state=8, l2_regularization=0.0020612244897959186, learning_rate=0.08336734693877551,
  max_depth=17, max_leaf_nodes=20, min_samples_leaf=28))])

pipexg = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),('estimator', xgb.XGBClassifier(random_state=0, learning_rate=0.083, max_depth=5,colsample_bytree=0.75, min_child_weight=4, reg_alpha=0.001))])

piperandom = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),('estimator', RandomForestClassifier(random_state=3, max_depth=12, n_estimators=325, min_samples_split=22, max_features=7))])

pipelg = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),('estimator', lgb.LGBMClassifier(n_estimators=300, max_depth=2, reg_alpha=3, reg_lambda=2, colsample_bytree=0.7))])

pipegb = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),('estimator', GradientBoostingClassifier(random_state=3, max_depth=2, n_estimators=600, min_samples_split=5, max_features=2, learning_rate=0.11))])

estimators = [
('1', xgb.XGBClassifier(random_state=0, learning_rate=0.083, max_depth=5,colsample_bytree=0.75, min_child_weight=4, reg_alpha=0.001)),
('2', HistGradientBoostingClassifier(random_state=8, l2_regularization=0.0020612244897959186, learning_rate=0.08336734693877551,
  max_depth=17, max_leaf_nodes=20, min_samples_leaf=28)),
('3', piperandom),
('4', pipegb),
('5',  lgb.LGBMClassifier(n_estimators=300, max_depth=2, reg_alpha=3, reg_lambda=2, colsample_bytree=0.7))]
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(max_iter=300), n_jobs=-1)

PARAMETERS = {#'2__max_iter': [150, 200],
#"colsample_bytree":[0.71, 0.75, 0.73, 0.74, 0.76],
#"1__max_depth": [5]
#"min_child_weight":[4, 5, 6, 7, 8, 9],
#"learning_rate":[ 0.088, 0.083]}
}

#model = GridSearchCV(clf, param_grid=PARAMETERS, cv=5, n_jobs=-1, scoring="accuracy")

#model.fit(train_df, train_target)

scores = cross_val_score(clf, train_df, train_target, cv=5, n_jobs=-1)

In [125]:
scores.mean()

0.6002642674662798

In [136]:
pipehist = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),('estimator', HistGradientBoostingClassifier(random_state=8, l2_regularization=0.0020612244897959186, learning_rate=0.08336734693877551,
  max_depth=17, max_leaf_nodes=20, min_samples_leaf=28))])

pipexg = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),('estimator', xgb.XGBClassifier(random_state=0, learning_rate=0.083, max_depth=5,colsample_bytree=0.75, min_child_weight=4, reg_alpha=0.001))])

piperandom = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),('estimator', RandomForestClassifier(random_state=3, max_depth=12, n_estimators=325, min_samples_split=22, max_features=7))])

pipelg = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),('estimator', lgb.LGBMClassifier(n_estimators=300, max_depth=2, reg_alpha=3, reg_lambda=2, colsample_bytree=0.7))])

pipegb = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),('estimator', GradientBoostingClassifier(random_state=3, max_depth=2, n_estimators=600, min_samples_split=5, max_features=2, learning_rate=0.11))])

estimators = [
('1', xgb.XGBClassifier(random_state=0, learning_rate=0.083, max_depth=5,colsample_bytree=0.75, min_child_weight=4, reg_alpha=0.001)),
('2', HistGradientBoostingClassifier(random_state=8, l2_regularization=0.0020612244897959186, learning_rate=0.08336734693877551,
  max_depth=17, max_leaf_nodes=20, min_samples_leaf=28)),
('3', piperandom),
('4', pipegb),
('5',  lgb.LGBMClassifier(n_estimators=300, max_depth=2, reg_alpha=3, reg_lambda=2, colsample_bytree=0.7))]
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(max_iter=300), n_jobs=-1)


clf.fit(train_df, train_target)
clf.score(train_df, train_target)

0.7278335252508636

In [269]:
scores

array([0.58840461, 0.59827303, 0.59498355, 0.58823529, 0.61456191])

La puntuación final sería la media de las cinco.

Y finalmente aplico el modelo entrenado (ignorando id):

In [137]:
predictions = clf.predict(test_df)
predictions = targetencoder.inverse_transform(predictions)
print(predictions)

['Acustico/Folk' 'Rock' 'Blues' ... 'Rock' 'Rock' 'Rock']


Ahora que tengo las predicciones voy a crear el fichero

In [138]:
submission = pd.DataFrame({'id': ids, 'Expected': predictions})

In [139]:
submission

,id,Expected
0,12760,Acustico/Folk
1,10607,Rock
2,4355,Blues
3,757,Alt
4,11503,Blues
...,...,...
3594,396,Acustico/Folk
3595,12017,Rock
3596,6520,Rock
3597,5591,Rock


Vemos que tenemos el tamaño adecuado:

In [140]:
submission.shape

(3599, 2)

Creamos el fichero del modelo:

In [141]:
submission.to_csv("tuned60val.csv", index=False)

In [142]:
from collections import Counter
print(Counter(predictions))

Counter({'Rock': 1429, 'Acustico/Folk': 497, 'Alt': 400, 'Metal': 348, 'Rap': 298, 'Blues': 190, 'Bollywood': 149, 'Instrumental': 126, 'Country': 76, 'Alt_Music': 73, 'Indie': 13})


In [276]:
from collections import Counter
print(Counter(predictions))

Counter({'Rock': 1420, 'Acustico/Folk': 519, 'Alt': 388, 'Metal': 349, 'Rap': 303, 'Blues': 188, 'Bollywood': 154, 'Instrumental': 125, 'Country': 76, 'Alt_Music': 66, 'Indie': 11})


In [228]:
print(Counter(clf.predict(test_df)))

Counter({10: 1485, 0: 488, 1: 439, 8: 305, 9: 287, 3: 175, 4: 152, 7: 123, 5: 73, 2: 70, 6: 2})


In [231]:
print(Counter(calibrated_clf.predict(test_df)))

Counter({10: 1490, 0: 524, 1: 442, 9: 284, 8: 247, 3: 200, 4: 150, 7: 123, 5: 70, 2: 69})
